Example case study to demonstrate the use of src.energiapy for solar panel reverse logistics network! Will mess around in SOLAR branch

In [12]:
import pandas 
import numpy

from src.energiapy.components.temporal_scale import TemporalScale
from src.energiapy.components.resource import Resource, VaryingResource
from src.energiapy.components.process import Process, VaryingProcess
from src.energiapy.components.material import Material
from src.energiapy.components.location import Location
from src.energiapy.components.transport import Transport
from src.energiapy.components.network import Network
from src.energiapy.components.scenario import Scenario
from src.energiapy.components.transport import Transport
from src.energiapy.components.result import Result 
from src.energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from src.energiapy.model.formulate import formulate, Constraints, Objective
from src.energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from src.energiapy.plot import plot_results, plot_scenario
from src.energiapy.plot.plot_results import CostY, CostX
from src.energiapy.model.solve import solve
from src.energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from src.energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit
from src.energiapy.aggregation.dtw import dynamic_warping, dynamic_warping_matrix, dynamic_warping_path, IncludeDTW
from src.energiapy.utils.data_utils import load_results
import matplotlib.pyplot as plt
from matplotlib import rc
from itertools import product

Initialize model

Temporal scale

In [13]:
scales = TemporalScale(discretization_list=[1, 365])

Resources

In [14]:
SiPV = Resource(name = 'sipv', label='Silicon Photovoltaic', price=50, basis = 'module', cons_max=130000)
glass = Resource(name = 'glass', label='Recycled glass', revenue=100, basis = 'kg', demand=True)
aluminum = Resource(name = 'al', label='Recycled aluminum', revenue=100, basis = 'kg', sell=True)
polymer = Resource(name = 'polymer', label='Recycled polymer', revenue=100, basis = 'kg', sell=True)
copper = Resource(name = 'cu', label='Recycled copper', revenue=100, basis = 'kg', sell=True)
silver = Resource(name = 'ag', label='Recycled silver', revenue=100, basis = 'kg', sell=True)
silicon = Resource(name = 'si', label='Recycled silicon', revenue=100, basis = 'kg', sell=True)

Processes

In [15]:
FRELP = Process(name = 'FRELP', label='FRELP recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.17945, silver: 0.001739}, prod_max = 1300, capex = 4, vopex = 8)
ASU = Process(name = 'ASU', label='ASU recycling', conversion = {SiPV: -1, glass: 13.73625, polymer: 2.22, aluminum: 1.5651, silicon: 0.4995, copper: 0.15355, silver: 0.001369}, prod_max = 1300, capex = 17, vopex = 9)
hybrid = Process(name = 'hybrid', label='Hybrid recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.15355, silver: 0.001369}, prod_max = 1300, capex = 3, vopex = 7)


FRELP_big = Process(name = 'FRELP_big', label='big FRELP recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.17945, silver: 0.001739}, prod_max = 13000, capex = 3, vopex = 5)
ASU_big = Process(name = 'ASU_big', label='big ASU recycling', conversion = {SiPV: -1, glass: 13.73625, polymer: 2.22, aluminum: 1.5651, silicon: 0.4995, copper: 0.15355, silver: 0.001369}, prod_max = 13000, capex = 8, vopex = 6)
hybrid_big = Process(name = 'hybrid_big', label='big Hybrid recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.15355, silver: 0.001369}, prod_max = 13000, capex = 3, vopex = 4)


Location

In [16]:
mitsubishi = Location(name='mitsubishi', processes = {FRELP, ASU, hybrid, FRELP_big, ASU_big, hybrid_big}, scales = scales, label = 'Mitsubishi recycling')

# houston = Location(name = 'HO', scales = scales, , processes = {dummy}, label = 'Houston')

Scenario

In [17]:
glassDelivery = Scenario(name = 'glassDelivery', network = mitsubishi, demand = {mitsubishi: {glass: 15000}}, scales= scales, label = 'glassDelivery', network_scale_level = 0, purchase_scale_level = 1, scheduling_scale_level = 1, demand_scale_level = 1, expenditure_scale_level = 0)

MILP formulation

In [18]:
milp_demand = formulate(scenario = glassDelivery, constraints={Constraints.INVENTORY,Constraints.COST, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE}, objective=Objective.MAX_DISCHARGE, objective_resource=glass)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint location vopex
constraint location incidental
constraint network capex
constraint network fopex
constraint network vopex
constraint network incidental
constraint nameplate inventory
constraint storage max
constraint storage min
constraint production mode
constraint nameplate production
constraint production max
constraint production min
constraint inventory balance
constraint resource consumption
constraint resource purchase
constraint location production
constraint location discharge
constraint location consumption
constraint location purchase
constraint network production
constraint network discharge
constraint network consumption
constraint network purchase
constraint demand
constraint network cost
objective discharge max


In [20]:
results = solve(scenario = glassDelivery, instance=milp_demand, solver='gurobi', print_solversteps=True, name = f'results')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\NCHRIS~1\AppData\Local\Temp\tmpjbk68xj0.log
   Prev:   Default: 
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8103 rows, 7367 columns and 33347 nonzeros
Model fingerprint: 0x8d879938
Coefficient statistics:
  Matrix range     [1e-03, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 8103 rows and 7367 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1364066e+08   0.000000e+00   0.000000e+00      0s

Solved with dual simplex
Solved in 0 it